In [1]:
import findspark
findspark.init()

from tokenize import String
import pyspark
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext, Row
from pyspark.sql.functions import *
import json
import os

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

22/11/10 20:43:38 WARN Utils: Your hostname, razer-arch resolves to a loopback address: 127.0.1.1; using 10.0.0.243 instead (on interface wlo1)
22/11/10 20:43:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark = (
    SparkSession.builder.appName("Phone Book - Country Look up")
    .config("spark.some.config.option", "some-value")
    .config("spark.sql.caseSensitive", "false")
    .getOrCreate()
)
spark.conf.set("spark.sql.caseSensitive", False)

In [4]:
DF2 = spark.read.json("cityStateMap.json")
DF1 = spark.read.json("tweets.json")

DF1.show()
DF2.show()

+-------------+--------------------+-------+
|          geo|               tweet|   user|
+-------------+--------------------+-------+
|      Atlanta|  It is a sunny day!|    Bob|
|       Athens|We have a footbal...|  Susan|
|      Atlanta|      Today is cold.|  David|
|       Auburn|I love Auburn Uni...|   Lisa|
|   Birmingham|I will go to Atla...|    Ben|
|San Francisco|We watch a movie ...|   Paul|
|    San Diego|It is hot today. ...|  Smith|
|  Log Angeles|Oscar ceremony is...|  Ethan|
|  Log Angeles|I love Oscar cere...|   Emma|
|      Orlando|I will go to the ...|Rolando|
|        Miami|          Sunny Day!|    Mia|
+-------------+--------------------+-------+

+-------------+----------+
|         city|     state|
+-------------+----------+
|      Atlanta|   Georgia|
|       Athens|   Georgia|
|        Miami|   Florida|
|      Orlando|   Florida|
|   Birmingham|   Alabama|
|       Auburn|   Alabama|
|  Log Angeles|California|
|San Francisco|California|
|    San Diego|California|


In [5]:
# 1) print only tweets from atlanta
# used filter here, to verify it geo is atlanta and
# then used select function for displaying only tweet.
DF3 = DF1.filter("geo == 'Atlanta'").select(["tweet", "geo"]).show()

+------------------+-------+
|             tweet|    geo|
+------------------+-------+
|It is a sunny day!|Atlanta|
|    Today is cold.|Atlanta|
+------------------+-------+



In [6]:
# 2) print only tweets that contain the word "today"
DF5 = DF1.filter((DF1.tweet.contains("today")) | (DF1.tweet.contains("Today"))).show()

+-------------+--------------------+-----+
|          geo|               tweet| user|
+-------------+--------------------+-----+
|       Athens|We have a footbal...|Susan|
|      Atlanta|      Today is cold.|David|
|   Birmingham|I will go to Atla...|  Ben|
|San Francisco|We watch a movie ...| Paul|
|    San Diego|It is hot today. ...|Smith|
+-------------+--------------------+-----+



In [7]:
# 3) Print only tweets from california
# first join is used, as state is in DF2.
# and then as geo and city are same, dropped a column,
# and now used filter to check if state is california and then displayed only tweet column
DF1.join(DF2, DF1.geo == DF2.city, "inner").drop(DF2.city).show()
DF6 = DF1.join(DF2, DF1.geo == DF2.city, "inner").drop(DF2.city)
DF6.filter("state == 'California'").select(["tweet", "state"]).show()

+-------------+--------------------+-------+----------+
|          geo|               tweet|   user|     state|
+-------------+--------------------+-------+----------+
|      Atlanta|  It is a sunny day!|    Bob|   Georgia|
|       Athens|We have a footbal...|  Susan|   Georgia|
|      Atlanta|      Today is cold.|  David|   Georgia|
|       Auburn|I love Auburn Uni...|   Lisa|   Alabama|
|   Birmingham|I will go to Atla...|    Ben|   Alabama|
|San Francisco|We watch a movie ...|   Paul|California|
|    San Diego|It is hot today. ...|  Smith|California|
|  Log Angeles|Oscar ceremony is...|  Ethan|California|
|  Log Angeles|I love Oscar cere...|   Emma|California|
|      Orlando|I will go to the ...|Rolando|   Florida|
|        Miami|          Sunny Day!|    Mia|   Florida|
+-------------+--------------------+-------+----------+

+--------------------+----------+
|               tweet|     state|
+--------------------+----------+
|We watch a movie ...|California|
|It is hot today. ...|C

In [8]:
# 4) Count the number of tweets published in each state.
# it is required to count the tweets, so we need to group by state to get count
DF8 = DF6.groupBy("state").count().show()

+----------+-----+
|     state|count|
+----------+-----+
|   Georgia|    3|
|   Alabama|    2|
|   Florida|    2|
|California|    4|
+----------+-----+

